# Lesson Learned, this had a private scored of 0.41 between Sliver to Bronze score ranges, if I had just let it pick the best two scores.

# TRY old school RFC and XGBoost on Gamma to see if it outperforms the current MLP on training CSV (which has a .2 ish, once scored .17).

# ICR Age Related Conditions Prediction of Class 1 (has one or more conditions, or Class 0 does not have conditions)
# Author, Thomas Gamet, released under the Apache 2.0 license

Not yet using greeks and currently filling in Null and NaN with standard deviation and mean based normalization.
Three models were experimented with:
1. Multi Layer Perceptron - seems the most promising at 128 and 64 neuron layers
2. Convolutional Neural Network - several attempts made, underperformed MLP
3. Long Short Term Memory Neural Network - underperformed MLP

There are two versions of the balanced_log_loss implemented for tensor flow.
One works with MLP and CNN which has dimensionality 2 for y predictions.
The other is for LSMT which uses dimensionality 3 for y predictions.

Key notes:
1. Tuning with drops of weights causes division by 0 and out of bounds log calculations, just don't bother.
2. All batch sizes are set to all training data so that we never run without having both class 1 and class 0. 
2.1. If a batch is missing all of one or the other you'll get a 0 in N0 or N1 which will cause problems.
2.2. I have not experimented with creating my own stratified batches.
3. The balanced_log_loss was updated to follow guidance from the Contest page.
3.1. It was further updated on 6/24/2023 to work for N>=2 categories.

Plan:
1. See which model shows best initial promise - so far MLP (two layers, recommending between 90 and 128 nodes - currently running with 128 in the 1st layer, and 64 in the second layer).
2. See if means and stddev do better than 0.0 in filling NaN values in features (done and yes).
3. Can greeks.csv help do a better job of feature preparation (maybe)?
4. SMOTE is non-trivial, using impbalance over_sample module to generate synthetic numeric features, then tuning validation data selection, epochs, and features. SMOTE did not work out, scores became 2x or more times worse.
5. Observed that the Greek's data for Gamma maps to Class 0 for Gamma categories M and N, and to class 1 for all other 6 categories. 
5.1. First pass to see if Gamma can provide a corrective prediction failed.
5.2. I am considering alternative approaches to Classifying Gamma (until its classification outperforms the train.csv set it is likely not yet suitable for a voter role).
6. Are the features causing any confusion - do we have too many (maybe, we are trying 14 features based on MIscore>0.04 (based on analysis of 9 with MIscore>0.05 and 19 with MIscore>0.03 not being significantly better)).
6.1. Possibly over tuning on 6/3/2023 - added highest 38 MI scores less 'AH' which caused log loss to increase (become worse) as features from 33 to 38 were added.
6.2. May search the first 33 features for any which when removed improve the internal balanced log loss???
7. Do we need stability - make an ensemble work for just MLP (running 5 times, but instead of averaging I'm going to test if the ends or middle are best).
8. Tune MLP.
8.1. Tracking this in the MLP declaration and new MI scoring code section
8.2. Competion is focused on Public score which is with the real validation set
8.3. Must use all training data to optimize Public score and graph its success, however for stopping conditions a good split was determined for 0.05 testing data.
8.4. Tune the model after finding cross over of training loss against public score
8.5. A voter for the 5 selected models (starting at basemodel) was written and the hope is that best probability can be chosen from the 3 to 5 agreeing votes based upon the size of the majority vote (unanimous uses the highest probability, 4 of 5 majority vote uses the second highest probability, and a simple vote uses the middle (also second of 3 of 5) probabilities.) 
9. Try to tune CNN also - notes left elsewhere, it under performed MLP
10. Try to tune LSTM also - the long short term memory does not seem to help - and the scores are less favorable than MLP
11. See if an ensemble of the two best models helps. Yes, we are voting on an ensemble of 5.

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
submission_df = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
print(train.head())
print(greeks.head())
print(test.head())
print(train.describe())
print(greeks.describe())
print(test.describe())

NameError: name 'pd' is not defined

# Normalize the training data before reorganizing it for batch processing

Study of Greeks

The standardization in the next two code blocks came from Chris Deotte.
It is a clean implementation that ensures the standardization used in
training is also used on the test data.

In [ ]:
train['EJ'] = train.EJ.map({'A':0.0,'B':1.0})
train['EJ'].astype(float)

FEATURES = []
# SAVE MEANS, STDS, NANS FOR TEST INFERENCE LATER
means = {}; stds = {}; nans = {}

for c in train.columns[1:-1]:
    m = train[c].mean()
    means[c] = m
    s = train[c].std()
    stds[c] = s
    train[c] = (train[c]-m)/s
    n = train[c].min() - 0.5
    nans[c] = n
    train[c] = train[c].fillna(n)
    FEATURES.append(c)
    
print(train.describe())
print(train.head())

# scores on features used became worse, the synthetic data produced herein
# is not conducive / helpful, if anything it appears to be harmful.

# WARNING: If we stop dropping column 'Id' here, then we must uncomment where 'Id' is dropped further on
# This new code block is attempting to use the text book SMOTE which hopefully helps get closer to a quality score

from imblearn.over_sampling import SMOTE # fortunately this is already pip installed or otherwise in the Docker image

strain = train.copy()
otrain = train.copy()

def applySMOTE(df):
    #numeric_columns = df.select_dtypes(include=[float,int]).columns
    #df_numeric = df[numeric_columns].copy()
    #df['Class'] = df['Class'].fillna(0)
    y = df['Class']  # Target variable
    X = df.drop('Class', axis=1)  # Features (excluding the target variable)
    X = df.drop('Id', axis=1)  # Features (excluding the target variable)

    # Apply SMOTE to generate synthetic samples
    smote = SMOTE(sampling_strategy=0.8, random_state=92833)  # 80% of the minority class size
    X_smote, y_smote = smote.fit_resample(X, y)

    # Combine the synthetic samples with the original DataFrame
    #smotedf = pd.concat([pd.DataFrame(X_smote), pd.DataFrame(y_smote, columns=['Class'])], axis=1)

    return X_smote

train = applySMOTE(strain)

print(train.describe())
print(train.head())
print(train.tail())

In [ ]:
# Greeks Alpha is A for class 0, and B, D, or G for class 1
# Beta, Gamma, Delta would need to be predicted from training data and then
# their predictions may be used to determine class 0 and class 1 by predicting
# Alpha and collapsing Alpha as A to class 0, and Alpha as B, D, G to class 1.
# We can also use Alpha alone to diagnose Class 0, and the three types of Class 1.

# First step, make sure Greeks data remains inorder with training data
# when re-arranged.

train['greek_alpha'] = greeks['Alpha']
train['greek_beta'] = greeks['Beta']
train['greek_gamma'] = greeks['Gamma']
train['greek_delta'] = greeks['Delta']

print(train.head())
print(train.describe())

In [ ]:
# original reorg to intermix 509 class 0 with 108 class 1 rows
strain = train.copy()
train = train.drop(train.index) # we want all the columns, but none of the data
# now, we are going to refill train, but with a guarantee that every 6th or more
# rows has at least one class 1 member
strain = strain.sort_values('Class', ascending=True) 
# this should leave us with 509 rows at for class 0 followed by 108 rows of class 1
countdown = 616 # last row is first class 1 to use
excess = 5 # 108 - int(509 / 5) - 1
for row in range(509):
    if row % 5 == 0:
        train = train.append(strain.iloc[countdown], ignore_index=True)
        countdown -= 1
        if row % 10 == 0 and excess > 0: 
            train = train.append(strain.iloc[countdown], ignore_index=True)
            countdown -= 1            
            excess -= 1
    train = train.append(strain.iloc[row], ignore_index=True)
train = train.append(strain.iloc[countdown], ignore_index=True) # the very last class 1

print(train.describe())
print(train.head())
print(train.tail())

In [ ]:
# Make a copy of the Greek data in the same order as the training data
# also pull it out of the dataframe so the existing code will work as designed

y_greeks = pd.DataFrame(train,columns=['greek_alpha', 'greek_beta', 'greek_gamma', 'greek_delta'])

train.drop('greek_alpha', axis=1, inplace=True)
train.drop('greek_beta', axis=1, inplace=True)
train.drop('greek_gamma', axis=1, inplace=True)
train.drop('greek_delta', axis=1, inplace=True)

print(y_greeks.describe())
print(y_greeks)

In [ ]:
# We perform the exact same feature engineering that we did with train. Specifically we standardize using the same means, stds, and nans.
test['EJ'] = test.EJ.map({'A':0,'B':1})
for c in test.columns[1:]:
    m = means[c]
    s = stds[c]
    test[c] = (test[c]-m)/s
    n = nans[c]
    test[c] = test[c].fillna(n)
    
print(test.describe())
print(test.head())

In [ ]:
# select numerical and categorical variables respectively.
num_cols = train.select_dtypes(include=['float64']).columns.tolist()
cat_cols = train.select_dtypes(include=['object']).columns.tolist()
cat_cols.remove('Id') #already removed by SMOTE prep work above
print ("numeric columns", num_cols)
print ("categorical columns", cat_cols)

In [ ]:
all_df = train #pd.concat([train, test])
print(all_df[num_cols].hist())

In [ ]:
import random
import numpy as np
import os
import tensorflow as tf

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    
seed_everything(14242) # best try to make runs reproducible - can also tweak this as it affects training splits

In [ ]:

# the following works for MLP (Multi Layer Perceptron) and CNN (Convolutional Neural Networks)
import tensorflow as tf

def balanced_log_loss(y_true, y_pred):
    num_classes = 8

    # Calculate the number of observations for each class.
    N = tf.reduce_sum(y_true, axis=0)

    # Calculate the weights for each class (inverse prevalence).
    weights = 1.0 / N

    # Calculate the predicted probabilities for each class.
    p = tf.clip_by_value(y_pred, 1e-15, 1 - 1e-15)

    # Calculate the log loss for each class.
    log_losses = -tf.reduce_sum(y_true * tf.math.log(p), axis=0)

    # Calculate the balanced logarithmic loss.
    balanced_log_loss = tf.reduce_mean(weights * log_losses)

    return balanced_log_loss

#def balanced_log_loss(y_true, y_pred):
#    # Calculate the number of observations for each class.
#    N_0 = tf.reduce_sum(y_true[:,0])
#    N_1 = tf.reduce_sum(y_true[:,1])
#
#    # Calculate the weights for each class. Thi  is the inverse [of the] prevalence.
#    w_0 = 1 / N_0 #N_1 / (N_1 + N_0) #1 / N_0
#    w_1 = 1 / N_1 #N_0 / (N_1 + N_0) #1 / N_1
#
# Calculate the predicted probabilities for each class.
#    p_0 = tf.clip_by_value(y_pred[:, 0], 1e-15, 1 - 1e-15)
#    p_1 = tf.clip_by_value(y_pred[:, 1], 1e-15, 1 - 1e-15)
#
#    # Calculate the log loss for each class.
#    log_loss_0 = -w_0 * tf.reduce_sum(y_true[:,0] * tf.math.log(p_0))
#    log_loss_1 = -w_1 * tf.reduce_sum(y_true[:,1] * tf.math.log(p_1))
#
#    # Calculate the balanced logarithmic loss.
#    balanced_log_loss = (log_loss_0 + log_loss_1) / 2.0
#
#    return balanced_log_loss

# the following works for LSTM (Long Short Term Memory) neural networks
def balanced_log_loss_lstm(y_true, y_pred):
    # Calculate the number of observations for each class.
    N_0 = tf.reduce_sum(y_true[:,0])
    N_1 = tf.reduce_sum(y_true[:,1])

    # Calculate the weights for each class.
    w_0 = 1 / N_0 #N_1 / (N_1 + N_0) #1 / N_0
    w_1 = 1 / N_1 #N_0 / (N_1 + N_0) #1 / N_1

    # Calculate the predicted probabilities for each class.
    p_0 = tf.clip_by_value(y_pred[:, 0, 0], 1e-15, 1 - 1e-15)
    p_1 = tf.clip_by_value(y_pred[:, 0, 1], 1e-15, 1 - 1e-15)

    # Calculate the log loss for each class.
    log_loss_0 = -w_0 * tf.reduce_sum(y_true[:,0] * tf.math.log(p_0))
    log_loss_1 = -w_1 * tf.reduce_sum(y_true[:,1] * tf.math.log(p_1))

    # Calculate the balanced logarithmic loss.
    balanced_log_loss = (log_loss_0 + log_loss_1) / 2

    return balanced_log_loss


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import make_pipeline

#numerical_features = ['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', #'BC', 
#                      'BD', 'BN', 'BP', 'BQ', 'BR', 'BZ',
#                      'CB', 'CC', 'CD', 'CF', 'CH', #'CL', 
#                      'CR', 'CS', 'CU', 'CW',
#                      'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
#                      'EB', 'EE', 'EG', 'EH', 'EL', 'EP', 'EU',
#                      'FC', 'FD', 'FE', 'FI', 'FL', 'FR', 'FS',
#                      'GB', 'GE', 'GF', 'GH', 'GI', 'GL']
#categorical_features = ['EJ']
#features = numerical_features + categorical_features
#str2int_dict = {}
#str2int_dict['EJ'] = {'A': 1, 'B': 0}

y = pd.DataFrame(all_df['Class'],columns=['class_0'])
y['class_1'] = all_df['Class'].fillna(0)
y['class_0'] = y['class_1'] * -1 + 1

print(y)

X_transformed = all_df.copy()
X_transformed.drop('Id', axis=1, inplace=True) #already removed by SMOTE above
y_mi = X_transformed['Class']
X_transformed.drop('Class', axis=1, inplace=True)

y = y.astype('float')
y_mi = y_mi.copy().astype('int')
print(y_mi) 

print(X_transformed.shape)
y = np.array(y) #.reshape(-1,1)
print(y.shape)

##Mutual Information Scoring

In [ ]:
from sklearn.feature_selection import mutual_info_classif
import pandas as pd

# the following runs were on 1000 epochs with full training set in use
# Setting 0.05 as MI score results in  9 features estimated loss .16 accuracy ~75%
# Seeting 0.04 as MI score results in 14 features estimated loss .13 accuracy ~85%
# Setting 0.03 as MI score results in 19 features estimated loss .08 accuracy ~85%

# the following runs were on 1000 epochs with 20% validation
# Setting 0.05 as MI score results in  9 features estimated loss .17 accuracy ~75%
# Seeting 0.04 as MI score results in 14 features estimated loss .17 accuracy ~75%
# Setting 0.03 as MI score results in 19 features estimated loss .18 accuracy ~75%

def rank_features_by_mutual_info(tdf: pd.DataFrame, y: pd.Series):
    # Calculate the mutual information scores
    #mi_scores = tf.keras.metrics.mutual_information(tdf, y)
    mi_scores = mutual_info_classif(tdf, y, random_state=4112)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=tdf.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return list(mi_scores.items())

def top_features(feature_scores, outselector):
    top_features_list = []
    for feature in feature_scores:
        if feature[1] > 0.01 or len(top_features_list) < outselector: # used 0.01 and 34 minimum before SMOTE
            top_features_list.append(feature[0])
        else:
            if len(top_features_list) == 0:
                   top_features_list.append(feature[0])
            break
    return top_features_list

totalfeaturesseen = 0
miscores = rank_features_by_mutual_info(X_transformed, y_mi)
print(miscores)
usefeats = top_features(miscores, 28)
#usefeats.remove('BR') # this is the 32nd and removing it decreases loss and increases accuracy
##usefeats.remove('GE') # this is the 31st and removing it increases loss and decreases accuracy
#usefeats.remove('EU') # this is the 30th and removing it decreases loss and increases accuracy
#usefeats.remove('AY') # this is the 29th and removing it decreases loss and increases accuracy
##usefeats.remove('AX') # this is the 28th and removing it increases loss and decreases accuracy
#usefeats.remove('FI') # this is the 27th and removing it decreases loss and decreases accuracy
##usefeats.remove('CD ')# this is the 26th and removing it aboutsame loss and aboutsame accuracy
##usefeats.remove('AR') # this is the 25th and removing it increased loss and increases accuracy
#usefeats.remove('CF') # this is the 24th and removing it decreases loss and increases accuracy
#usefeats.remove('BP') # this is the 23th and removing it decreases loss and increases accuracy
#usefeats.remove('AH') # MIscore rank 37 (only get's removed if 37 or more are selected)
print("Total features =", len(usefeats))
print("Feaures to use =", usefeats)

X_transformed = X_transformed[usefeats]

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout, Conv1D, Flatten, Input, MaxPooling1D, concatenate, BatchNormalization

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, Input, MaxPooling1D, concatenate, BatchNormalization

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# MLP model original input(56)-dense(128)-dense(128)-Flatten-Dense(2) relu->softmax
# Date     , loss, eloss, patience, min_del, epochs/uses ep, val_acc
# 5/27/2023, 0.48,  0.2209,     0010, 0.00500, 000225/225    , ~60%
# 5/28/2023, 0.41,  0.1576,     0010, 0.00500, 000500/300-350, ~80%
# 5/29/2023, 0.32,  0.0850,     0010, 0.00050, 000500/500    , ~85%
# 5/30/2023, 0.29,  0.0335,     0010, 0.00010, 000750/750    , ~95%
# Reducing to MIscore>0.04 providing 14 features
# 5/31/2023, 0.38,  0.1251,     0010, 0.00010, 001000/1000   , ~82%
# 6/01/2023, 0.23,  0.1683,     0010, 0.00010, 001000/390    , ~93% MIs>0.01, 0.05test MLP128x128
# 6/02/2023, 0.22,  0.1386,     0100, 0.00001, 002500/580,   , ~93% MIs>0.01, 0.05test MLP128x64
# 6/03/2023, 0.18,  0.1205,     0100, 0.00001, 002500/400-800, ~97% MIs>0.01, 0.05test MLP128x64 with 5 model voter base 5 and 33 features
# 6/04/2023, 0.21,  0.0949,     0100, 0.00001, 002500/400-800, ~96% MIs>0.01, 0.05test MLP128x64 with 5 model voter base 5 and 37of38 features
# 6/05/2023, 0.18,  0.1140,     0100, 0.00001, 002500/400-800, ~96% MIs>0.01,5 and 33 fea 0.05test MLP128-160x64 with 5 model voter base features
# making it so about every 6th training row is class 1 allows batching 101
# 6/06/2023, 0.17,  0.1061,     0020, 0.00100, 002500/175-270, ~96% MIs>0.01, 0.05test MLP128-160x64 with 5 model voter base 5 and 34 features
# 6/07/2023, 0.19,  0.0865,     0020, 0.00100, 002500/175-270, ~97% MIs>0.01, 0.05test MLP128-160x64 with 5 model voter base 5 and 34 features less 'BR' batchsize=101
# 6/08/2023, 0.19,  0.0231,     0020, 0.00100, 002500/175-270, 100% MIs>0.01, 0.05test MLP128-160x64 with 5 model voter base 5 and 34 features less 'BR', 'EU', 'AY', 'FI, 'CF', and 'BP'
# 6/09/2023, 0.19,  0.0788,     0020, 0.00100, 002500/175-270, 100% MIs>0.01, 0.05test MLP128-160x64 with 5 model voter base 5 and 34 features less 'BR', 'EU', 'AY', 'FI, 'CF', and 'BP' stop near val_loss and loss crossing
# Either we need a better model or feature engineering - now trying SMOTE with 509 of each class
# 6/10/2023, 0.29,  0.0837,     0020, 0.00100, 000200, SMOTE ~98% MIs>0.01, 0.2test-unique MLP128-160x64 with 5 model voter base 5 batchsize 254
# 6/11/2023, 0.38, with SMOTE on 43 features with Dense len(usefeats)*1.5...
# 6/12/2023, 0.39, with SMOTE on 43 features with Dense 128x64
# 6/13/2023, 0.85, with SMOTE on 43 features with Dense len(usefeats) 
# The SMOTE approach appears to be failing. It is time to see what predictions
# the greeks.csv will produce (any one positive means class 1)
def MLPmodelBuilder(X_transformed):
    inputs = Input(shape=[X_transformed.shape[1]])
    dense = Dense(128, activation='relu')(inputs)
    #dense = Dropout(.20)(dense)
    dense = Dense(64, activation='relu')(dense)
    dense = Flatten()(dense)
    outputs = Dense(2, activation='softmax')(dense)
    return inputs, outputs

def MLPmodelBuilderOut3(X_transformed):
    inputs = Input(shape=[X_transformed.shape[1]])
    dense = Dense(128, activation='relu')(inputs)
    #dense = Dropout(.20)(dense)
    dense = Dense(64, activation='relu')(dense)
    dense = Flatten()(dense)
    outputs = Dense(3, activation='softmax')(dense)
    return inputs, outputs

def MLPmodelBuilderOut8(X_transformed):
    inputs = Input(shape=[X_transformed.shape[1]])
    dense = Dense(128, activation='relu')(inputs)
    #dense = Dropout(.20)(dense)
    dense = Dense(64, activation='relu')(dense)
    dense = Flatten()(dense)
    outputs = Dense(8, activation='softmax')(dense)
    return inputs, outputs

# Ran experimments with size 32 - 128 increments of 4
# Both val of .2 and .5 show that the larger nets do better, but 114 or 86 might do just as well
# Val scores suggest targets should be reached in the low 90% accuracy.
# With .5 val data, loss functions are 0.15 through val loss 0.33, which is not immpressive, but all training data should do a little better.
# Ran val set of .5 without Flattening, and it process a similar result to the Flatten calling version.
# Sigmoid DOES NOT PRODUCE probabilities we can use.
def MLPmodelBuilderSize(X_transformed, size):
    inputs = Input(shape=[X_transformed.shape[1]])
    dense = Dense(size, activation='relu')(inputs)
    #dense = Dropout(.20)(dense)
    dense = Dense(size/2, activation='relu')(dense)
    #dense = Dropout(.20)(dense)
    #dense = Dense(size/4, activation='relu')(dense) # seems to confuse results - overfit
    #dense = Flatten()(dense)
    outputs = Dense(2, activation='softmax')(dense) #tried sigmoid
    return inputs, outputs

# MLP model must try
# the highest score yielded a 0.32 (internal around .10) and lowest a 0.33 (internal 0.08)
#def MLPmodelBuilder(X_transformed):
#    inputs = Input(shape=[X_transformed.shape[1]])
#    dense = Dense(42, activation='relu')(inputs)
#    #dense = Dense(21, activation='relu')(dense)
#    #dense = Dense(114, activation='relu')(dense)
#    dense = Flatten()(dense)
#    outputs = Dense(2, activation='softmax')(dense)
#    return inputs, outputs

def CNNmodelBuilderSize(X_transformed, size):
    inputs = Input(shape=[X_transformed.shape[1], 1])
    conv = Conv1D(size, kernel_size=6, activation='relu')(inputs)
    conv = MaxPooling1D(pool_size=2)(conv)
    conv = Conv1D(size/2, kernel_size=6, activation='relu')(conv)
    conv = MaxPooling1D(pool_size=2)(conv)
    #conv = Conv1D(size/4, kernel_size=6, activation='relu')(conv)
    #conv = MaxPooling1D(pool_size=2)(conv)
    conv = Flatten()(conv)
    outputs = Dense(2, activation='softmax')(conv)  
    return inputs, outputs

def LSTMmodelBuilder(X_transformed):
    inputs = Input(shape=(1, X_transformed.shape[1]))
    # Let's try an LSTM
    lstm = tf.keras.layers.LSTM(len(X_transformed), 
                                recurrent_activation='relu',
                                activation='relu',
                                #dropout=.5,
                                #recurrent_dropout=.5,
                                return_sequences=True)(inputs)
    # define the output layer
    outputs = Dense(2, activation='softmax')(lstm)
    return inputs, outputs

# Custom build of neural networks

# due to the voter the minimum number of models is now 10, and 15 should be used.
# tupple with 0 = MLP, 1 = CNN, 2 = LSTM
# due to the voter, all models must be of the same type
models = []
for i in range(15):
    inputs, outputs = MLPmodelBuilder(X_transformed)
    #inputs, outputs = MLPmodelBuilderSize(X_transformed, 96+i*4)
    #inputs, outputs = CNNmodelBuilderSize(X_transformed, 32+i*2)
    #inputs, outputs = LSTMmodelBuilder(X_transformed)
    modelx = Model(inputs=inputs, outputs=outputs)
    models.append([modelx, 0]) # must change 0 to 2 for LSTM, and optional to 1 for CNN
    #inputs, outputs = CNNmodelBuilderSize(X_transformed, 128)
    #modelx = Model(inputs=inputs, outputs=outputs)
    #models.append([modelx, 1]) # must change 0 to 2 for LSTM, and optional to 1 for CNN

print("Models are built and ready to fit...")

In [ ]:
from keras.utils.vis_utils import plot_model
from tensorflow import keras
from sklearn.model_selection import train_test_split

# This code block compiles and build 15 models for 
# the main training set of class 0 and class 1
# The best answers are between 0.17 and 0.20 log loss

#maxepochs = [2500, 2500, 2500, 2500, 2500,  # probably overfits
#             2500, 2500, 2500, 2500, 2500,  # 0.1% test data 95% and 0.17ish
#             2500, 2500, 2500, 2500, 2500]  # 0.05% test at 98% and 0.05-0.09 batch size 202
#maxepochs = [230, 270, 140, 160,175,  # try this with 0.05% test data
#             265, 230, 275, 145,310,  # after the full run - 97% and 0.07-0.11
#             280, 220, 300, 145, 220]
#maxepochs = [100, 90, 80, 110, 120,   # try this with 0.1% test data
#             125, 120, 140, 75, 100,  # after full run 93% and .22-0.24
#             140, 125, 100, 75, 125]
maxepochs = [2000, 2000, 2000, 2000, 2000,  # probably overfits
             2000, 2000, 2000, 2000, 2000,  # 0.2% test data with SMOTE procedure for 509 in each class
             2000, 2000, 2000, 2000, 2000, 2000]  # 
devrun = True

# This was too hard to tune, especially without validation
early_stopping = keras.callbacks.EarlyStopping(
    patience=30, #18, # restore 18, # be very patient, long enough for the cross over
    min_delta=0.005,
    restore_best_weights=True,
)

# This was too hard to tune, especially without validation
early_stopping_lstm = keras.callbacks.EarlyStopping(
    patience=10,     # hyper tune these to get loss to stop near 0.12
    min_delta=0.0001,
    restore_best_weights=True,
)

#train , val = np.split(data.sample(frac = 1) , [int(len(data) * 0.8)])
#print(y)
#**** next experiment should be to find optimal split or see of smaller batches can work
X_train, X_val, y_train, y_val = train_test_split(X_transformed, y, test_size=0.05, random_state=42, stratify=y)

## Radical new approach, try tuning with all the data against just the 
## test with one run per day, and model improvements using early stopping
##
print ("Compiling models")

## notes when run with seed 14242
## MLP single model 1/1 [==============================] - 0s 40ms/step - loss: 21.1738 - accuracy: 0.9635 - binary_accuracy: 0.9635 - val_loss: 15.7028 - val_accuracy: 0.8952 - val_binary_accuracy: 0.8952
## CNN single model 1/1 [==============================] - 0s 83ms/step - loss: 24.7791 - accuracy: 0.9736 - binary_accuracy: 0.9736 - val_loss: 15.8823 - val_accuracy: 0.8468 - val_binary_accuracy: 0.8468
## LSTM single model 1/1 [==============================] - 0s 82ms/step - loss: 35.5633 - accuracy: 0.5000 - binary_accuracy: 0.6908 - val_loss: 16.8421 - val_accuracy: 0.5000 - val_binary_accuracy: 0.6613

modelno = 0
for modeltup in models:
    #X_train, X_val, y_train, y_val = train_test_split(X_transformed, y, test_size=0.2, random_state=42+(modelno%2)*100, stratify=y)
    model = modeltup[0]
    print ("Compiling and fitting model", modelno+1)
    tf.random.set_seed(91+modelno*79)
    #(optimizer='adam', loss = balanced_log_loss)
    if modeltup[1] == 2: # this is for LSTM
        model.compile(loss=balanced_log_loss_lstm,
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy', 'binary_accuracy'])
        Xd = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1])) # only for LSTM
        Xd_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1])) # only for LSTM
        history = model.fit(
            X_transformed, #Xd, 
            y, #y_train,
            epochs = 1500,
            batch_size = len(Xd),
            validation_data = (Xd_val, y_val),
            callbacks=[early_stopping_lstm]
        )
    else: # this covers both CNN and MLP
        model.compile(loss=balanced_log_loss,
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy', 'binary_accuracy'])
        history = model.fit(
            X_train, #X_transformed, #X_train, # for MLP and CNN, else Xd
            y_train, #y, #y_train,
            epochs = maxepochs[modelno], #2500, #225, 
            verbose = False, #devrun,
            batch_size = 101, # len(X_transformed),
            validation_data = (X_val, y_val),
            callbacks=[early_stopping]
        )
    modelno += 1
    if devrun:
        pd.DataFrame(history.history).plot()

In [ ]:
# Alternate Code Block 1 of 4
#
# This code block compiles and build 15 models for 
# the training the Greeks output for Beta 
# We also have blocks for Gamma and Delta
# Markdown this block to go back to the original or
# keep this 1 of 4 code blocks which predict Greeks and
# then use that prediction as features to predict class 0 and 
# class 1

maxepochs = [400, 400, 400, 400, 400,  # probably overfits
             400, 400, 400, 400, 400,  # 0.2% test data with SMOTE procedure for 509 in each class
             400, 400, 400, 400, 400, 400]  # 
devrun = True

# This was too hard to tune, especially without validation
early_stopping_g = keras.callbacks.EarlyStopping(
    patience=30, #18, # restore 18, # be very patient, long enough for the cross over
    min_delta=0.0005,
    restore_best_weights=True,
)

# This was too hard to tune, especially without validation
#early_stopping_lstm = keras.callbacks.EarlyStopping(
#    patience=10,     # hyper tune these to get loss to stop near 0.12
#    min_delta=0.0001,
#    restore_best_weights=True,
#)

gbmodels = []
for i in range(15):
    inputs, outputs = MLPmodelBuilderOut3(X_transformed)
    modelx = Model(inputs=inputs, outputs=outputs)
    gbmodels.append([modelx, 0]) # must change 0 to 2 for LSTM, and optional to 1 for CNN

print("Models are built and ready to fit...")

# rebuilt X_transformed for Greek Beta
yclone = y.copy()
y_beta = pd.DataFrame(yclone,columns=['A', 'B'])
y_beta['C'] = y_beta['B']
for row in range(len(y)):
    if y_greeks.loc[row,'greek_beta'] == 'A':
        y_beta.loc[row,'A'] = 1.0
        y_beta.loc[row,'B'] = 0.0
        y_beta.loc[row,'C'] = 0.0
    elif y_greeks.loc[row,'greek_beta'] == 'B':
        y_beta.loc[row,'A'] = 0.0
        y_beta.loc[row,'B'] = 1.0
        y_beta.loc[row,'C'] = 0.0
    else:
        y_beta.loc[row,'A'] = 0.0
        y_beta.loc[row,'B'] = 0.0
        y_beta.loc[row,'C'] = 1.0
print(y_beta.head())

X_train, X_val, y_train, y_val = train_test_split(X_transformed, y_beta, test_size=0.05, random_state=42, stratify=y)

## Radical new approach, try tuning with all the data against just the 
## test with one run per day, and model improvements using early stopping
##
print ("Compiling models")

## notes when run with seed 14242
## MLP single model 1/1 [==============================] - 0s 40ms/step - loss: 21.1738 - accuracy: 0.9635 - binary_accuracy: 0.9635 - val_loss: 15.7028 - val_accuracy: 0.8952 - val_binary_accuracy: 0.8952
## CNN single model 1/1 [==============================] - 0s 83ms/step - loss: 24.7791 - accuracy: 0.9736 - binary_accuracy: 0.9736 - val_loss: 15.8823 - val_accuracy: 0.8468 - val_binary_accuracy: 0.8468
## LSTM single model 1/1 [==============================] - 0s 82ms/step - loss: 35.5633 - accuracy: 0.5000 - binary_accuracy: 0.6908 - val_loss: 16.8421 - val_accuracy: 0.5000 - val_binary_accuracy: 0.6613

modelno = 0
for modeltup in gbmodels:
    #X_train, X_val, y_train, y_val = train_test_split(X_transformed, y, test_size=0.2, random_state=42+(modelno%2)*100, stratify=y)
    model = modeltup[0]
    print ("Compiling and fitting model", modelno+1)
    tf.random.set_seed(91+modelno*79)
    #(optimizer='adam', loss = balanced_log_loss)
    if modeltup[1] == 2: # this is for LSTM
        model.compile(loss=balanced_log_loss_lstm,
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy', 'binary_accuracy'])
        Xd = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1])) # only for LSTM
        Xd_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1])) # only for LSTM
        history = model.fit(
            X_transformed, #Xd, 
            y, #y_train,
            epochs = 1500,
            batch_size = len(Xd),
            validation_data = (Xd_val, y_val),
            callbacks=[early_stopping_lstm]
        )
    else: # this covers both CNN and MLP
        model.compile(loss=balanced_log_loss,
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy'])
        history = model.fit(
            X_train, #X_transformed, #X_train, # for MLP and CNN, else Xd
            y_train, #y, #y_train,
            epochs = maxepochs[modelno], #2500, #225, 
            verbose = False, #devrun,
            batch_size = 617, # len(X_transformed),
            validation_data = (X_val, y_val),
            #callbacks=[early_stopping_g]
        )
    modelno += 1
    if devrun:
        pd.DataFrame(history.history).plot()

In [ ]:
# Alternate Code Block 2 of 4 coded for LGBM
#
# This code block compiles and build 15 models for 
# the training the Greeks output for Beta 
# We also have blocks for Gamma and Delta
# Markdown this block to go back to the original or
# keep this 1 of 4 code blocks which predict Greeks and
# then use that prediction as features to predict class 0 and 
# class 1

from sklearn.metrics import log_loss, make_scorer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

def lgb_balanced_log_loss(y_true, y_pred):
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0))
    log_loss_1 = -np.sum(y_true * np.log(p_1))
    w_0 = 1 / N_0
    w_1 = 1 / N_1
    balanced_log_loss = 2*(w_0 * log_loss_0 + w_1 * log_loss_1) / (w_0 + w_1)
    return balanced_log_loss/(N_0+N_1)

def lgb_metric(y_true, y_pred):
    return 'balanced_log_loss', lgb_balanced_log_loss(y_true, y_pred), False

Xg_transformed = all_df.copy()

log_losses = []
balanced_log_losses = []

feature_importance_df_total = pd.DataFrame()

lgbmodels = {}  # key: bag, val: [model] * fold_num

lgbm_params = {"boosting_type": 'goss'
               , "learning_rate": 0.06733232950390658
               , "n_estimators": 50000
               , "early_stopping_round": 300
               , "random_state": 42
               , "subsample": 0.6970532011679706
               , "colsample_bytree": 0.6055755840633003
               , "class_weight": 'balanced'
               , "metric": 'none'
               , "is_unbalance": True
               , "max_depth": 8}

bag_num = 20
n_fold = 5

for bag in range(bag_num):
    
    print(f'########################## bag: {bag} ##########################')

    kf = StratifiedKFold(n_splits=n_fold, random_state=42 * bag, shuffle=True)
    
    lgbmodels[bag] = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(Xg_transformed, greeks['Gamma'])):
        # df.loc[test_idx, 'fold'] = fold

        train_df = Xg_transformed.iloc[train_idx]
        valid_df = Xg_transformed.iloc[test_idx]
        valid_ids = valid_df.Id.values.tolist()

        X_train, y_train = train_df[usefeats], train_df['Class']
        X_valid, y_valid = valid_df[usefeats], valid_df['Class']

        lgb = LGBMClassifier(**lgbm_params)

        lgb.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False,
                eval_metric=lgb_metric)
        
        # 
        feature_importances = lgb.feature_importances_
        # 
        feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Split': feature_importances, 'Gain': lgb.booster_.feature_importance(importance_type='gain')})
        feature_importance_df['bag'] = bag
        feature_importance_df['fold'] = fold
        
        feature_importance_df_total = pd.concat([feature_importance_df_total, feature_importance_df], axis=0)
        
        y_pred = lgb.predict_proba(X_valid)

        logloss = log_loss(y_valid, y_pred)
        balanced_logloss = lgb_balanced_log_loss(y_valid, y_pred[:, 1])
        log_losses.append(logloss)
        balanced_log_losses.append(balanced_logloss)
        
        lgbmodels[bag].append(lgb)

        print(f"Bags: {bag}, Fold: {fold}, log loss: {round(logloss, 3)}, balanced los loss: {round(balanced_logloss, 3)}")

print()
print("Log Loss")
# print(log_losses)
print(np.mean(log_losses), np.std(log_losses))
print()
print("Balanced Log Loss")
# print(balanced_log_losses)
print(np.mean(balanced_log_losses), np.std(balanced_log_losses))
print()

plt.hist(log_losses, bins=100) 
plt.axvline(x=np.mean(log_losses), color='red', linestyle='dashed', label='Mean: {:.2f}'.format(np.mean(log_losses))) 
plt.axvline(x=np.median(log_losses), color='blue', linestyle='dashed', label='Median: {:.2f}'.format(np.median(log_losses))) 
plt.legend() 
plt.title('Histogram of log_losses std: {:.2f}'.format(np.std(log_losses)),size=20) 
plt.show()

plt.hist(balanced_log_losses, bins=100) 
plt.axvline(x=np.mean(balanced_log_losses), color='red', linestyle='dashed', label='Mean: {:.2f}'.format(np.mean(balanced_log_losses))) 
plt.axvline(x=np.median(balanced_log_losses), color='blue', linestyle='dashed', label='Median: {:.2f}'.format(np.median(balanced_log_losses))) 
plt.legend() 
plt.title('Histogram of balanced_log_losses std: {:.2f}'.format(np.std(balanced_log_losses)),size=20) 
plt.show()



# Alternate Code Block 2 of 4
#
# This code block compiles and build 15 models for 
# the training the Greeks output for Beta 
# We also have blocks for Gamma and Delta
# Markdown this block to go back to the original or
# keep this 1 of 4 code blocks which predict Greeks and
# then use that prediction as features to predict class 0 and 
# class 1

maxepochs = [1000, 1000, 1000, 1000, 1000,  # probably overfits
             1000, 1000, 1000, 1000, 1000,  # 0.2% test data with SMOTE procedure for 509 in each class
             1000, 1000, 1000, 1000, 1000, 1000]  # 
devrun = True

# This was too hard to tune, especially without validation
early_stopping_g = keras.callbacks.EarlyStopping(
    patience=30, #18, # restore 18, # be very patient, long enough for the cross over
    min_delta=0.0005,
    restore_best_weights=True,
)

# This was too hard to tune, especially without validation
#early_stopping_lstm = keras.callbacks.EarlyStopping(
#    patience=10,     # hyper tune these to get loss to stop near 0.12
#    min_delta=0.0001,
#    restore_best_weights=True,
#)

ggmodels = []
for i in range(15):
    inputs, outputs = MLPmodelBuilderOut8(X_transformed)
    modelx = Model(inputs=inputs, outputs=outputs)
    ggmodels.append([modelx, 0]) # must change 0 to 2 for LSTM, and optional to 1 for CNN

print("Models are built and ready to fit...")

# rebuilt X_transformed for Greek Beta
yclone = y.copy()
y_gamma = pd.DataFrame(yclone,columns=['G', 'H'])
y_gamma['E'] = y_gamma['H']
y_gamma['F'] = y_gamma['H']
y_gamma['A'] = y_gamma['H']
y_gamma['B'] = y_gamma['H']
y_gamma['M'] = y_gamma['H']
y_gamma['N'] = y_gamma['H']
for row in range(len(y)):
    if y_greeks.loc[row,'greek_gamma'] == 'G':
        y_gamma.loc[row,'G'] = 1.0
        y_gamma.loc[row,'H'] = 0.0
        y_gamma.loc[row,'E'] = 0.0
        y_gamma.loc[row,'F'] = 0.0
        y_gamma.loc[row,'A'] = 0.0
        y_gamma.loc[row,'B'] = 0.0
        y_gamma.loc[row,'M'] = 0.0
        y_gamma.loc[row,'N'] = 0.0
    elif y_greeks.loc[row,'greek_gamma'] == 'H':
        y_gamma.loc[row,'G'] = 0.0
        y_gamma.loc[row,'H'] = 1.0
        y_gamma.loc[row,'E'] = 0.0
        y_gamma.loc[row,'F'] = 0.0
        y_gamma.loc[row,'A'] = 0.0
        y_gamma.loc[row,'B'] = 0.0
        y_gamma.loc[row,'M'] = 0.0
        y_gamma.loc[row,'N'] = 0.0
    elif y_greeks.loc[row,'greek_gamma'] == 'E':
        y_gamma.loc[row,'G'] = 0.0
        y_gamma.loc[row,'H'] = 0.0
        y_gamma.loc[row,'E'] = 1.0
        y_gamma.loc[row,'F'] = 0.0
        y_gamma.loc[row,'A'] = 0.0
        y_gamma.loc[row,'B'] = 0.0
        y_gamma.loc[row,'M'] = 0.0
        y_gamma.loc[row,'N'] = 0.0
    elif y_greeks.loc[row,'greek_gamma'] == 'F':
        y_gamma.loc[row,'G'] = 0.0
        y_gamma.loc[row,'H'] = 0.0
        y_gamma.loc[row,'E'] = 0.0
        y_gamma.loc[row,'F'] = 1.0
        y_gamma.loc[row,'A'] = 0.0
        y_gamma.loc[row,'B'] = 0.0
        y_gamma.loc[row,'M'] = 0.0
        y_gamma.loc[row,'N'] = 0.0
    elif y_greeks.loc[row,'greek_gamma'] == 'A':
        y_gamma.loc[row,'G'] = 0.0
        y_gamma.loc[row,'H'] = 0.0
        y_gamma.loc[row,'E'] = 0.0
        y_gamma.loc[row,'F'] = 0.0
        y_gamma.loc[row,'A'] = 1.0
        y_gamma.loc[row,'B'] = 0.0
        y_gamma.loc[row,'M'] = 0.0
        y_gamma.loc[row,'N'] = 0.0
    elif y_greeks.loc[row,'greek_gamma'] == 'B':
        y_gamma.loc[row,'G'] = 0.0
        y_gamma.loc[row,'H'] = 0.0
        y_gamma.loc[row,'E'] = 0.0
        y_gamma.loc[row,'F'] = 0.0
        y_gamma.loc[row,'A'] = 0.0
        y_gamma.loc[row,'B'] = 1.0
        y_gamma.loc[row,'M'] = 0.0
        y_gamma.loc[row,'N'] = 0.0
    elif y_greeks.loc[row,'greek_gamma'] == 'M':
        y_gamma.loc[row,'G'] = 0.0
        y_gamma.loc[row,'H'] = 0.0
        y_gamma.loc[row,'E'] = 0.0
        y_gamma.loc[row,'F'] = 0.0
        y_gamma.loc[row,'A'] = 0.0
        y_gamma.loc[row,'B'] = 0.0
        y_gamma.loc[row,'M'] = 1.0
        y_gamma.loc[row,'N'] = 0.0
    else:
        y_gamma.loc[row,'G'] = 0.0
        y_gamma.loc[row,'H'] = 0.0
        y_gamma.loc[row,'E'] = 0.0
        y_gamma.loc[row,'F'] = 0.0
        y_gamma.loc[row,'A'] = 0.0
        y_gamma.loc[row,'B'] = 0.0
        y_gamma.loc[row,'M'] = 0.0
        y_gamma.loc[row,'N'] = 1.0
print(y_gamma.head())

X_train, X_val, y_train, y_val = train_test_split(X_transformed, y_gamma, test_size=0.1, random_state=42, stratify=y)

## Radical new approach, try tuning with all the data against just the 
## test with one run per day, and model improvements using early stopping
##
print ("Compiling models")

## notes when run with seed 14242
## MLP single model 1/1 [==============================] - 0s 40ms/step - loss: 21.1738 - accuracy: 0.9635 - binary_accuracy: 0.9635 - val_loss: 15.7028 - val_accuracy: 0.8952 - val_binary_accuracy: 0.8952
## CNN single model 1/1 [==============================] - 0s 83ms/step - loss: 24.7791 - accuracy: 0.9736 - binary_accuracy: 0.9736 - val_loss: 15.8823 - val_accuracy: 0.8468 - val_binary_accuracy: 0.8468
## LSTM single model 1/1 [==============================] - 0s 82ms/step - loss: 35.5633 - accuracy: 0.5000 - binary_accuracy: 0.6908 - val_loss: 16.8421 - val_accuracy: 0.5000 - val_binary_accuracy: 0.6613

modelno = 0
for modeltup in ggmodels:
    #X_train, X_val, y_train, y_val = train_test_split(X_transformed, y, test_size=0.2, random_state=42+(modelno%2)*100, stratify=y)
    model = modeltup[0]
    print ("Compiling and fitting model", modelno+1)
    tf.random.set_seed(91+modelno*79)
    #(optimizer='adam', loss = balanced_log_loss)
    if modeltup[1] == 2: # this is for LSTM
        model.compile(loss=balanced_log_loss_lstm,
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy', 'binary_accuracy'])
        Xd = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1])) # only for LSTM
        Xd_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1])) # only for LSTM
        history = model.fit(
            X_transformed, #Xd, 
            y, #y_train,
            epochs = 1500,
            batch_size = len(Xd),
            validation_data = (Xd_val, y_val),
            callbacks=[early_stopping_lstm]
        )
    else: # this covers both CNN and MLP
        model.compile(loss=balanced_log_loss,
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy'])
        history = model.fit(
            X_train, #X_transformed, #X_train, # for MLP and CNN, else Xd
            y_train, #y, #y_train,
            epochs = maxepochs[modelno], #2500, #225, 
            verbose = False, #devrun,
            batch_size = 617, # len(X_transformed),
            validation_data = (X_val, y_val),
            #callbacks=[early_stopping_g]
        )
    modelno += 1
    if devrun:
        pd.DataFrame(history.history).plot()

Select one of the models for submission


Using MIscores on 0.05 test data and up to 2500 iterations, before using SMOTE, with MLP at size 128x64, here is the loss estimate

At MIscore 0.1 with Total features = 6
Feaures to use = ['DU', 'FL', 'BC', 'GL', 'BQ', 'AF']

Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.20610846885276007, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.3068458051364643, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 1ms/step
intermediate balanced log loss  tf.Tensor(0.22404914032602652, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 1ms/step
intermediate balanced log loss  tf.Tensor(0.23913039212074222, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.3335314783429701, shape=(), dtype=float64)

At MIscore 0.05 Total features = 9
Feaures to use = ['DU', 'FL', 'BC', 'GL', 'BQ', 'AF', 'FD ', 'DI', 'AB']

Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.13666516888732855, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.11567961657836084, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.09968245750887358, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.08188260623813551, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 1ms/step
intermediate balanced log loss  tf.Tensor(0.097783840724294, shape=(), dtype=float64)

At MIscore=0.01 Total features = 33
Feaures to use = ['DU', 'FL', 'BC', 'GL', 'BQ', 'AF', 'FD ', 'DI', 'AB', 'CC', 'EH', 'DA', 'FR', 'BN', 'AM', 'CR', 'GH', 'AY', 'EE', 'DF', 'FE', 'DE', 'CL', 'GF', 'FC', 'BP', 'CF', 'EB', 'EU', 'CD ', 'FI', 'GE', 'DH']

Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.1682810280943917, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.12137677262643709, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.18444699871451065, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.15090392797691776, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.1910776021856574, shape=(), dtype=float64)

At MIscore=0.005 Total features = 39
Feaures to use = ['DU', 'FL', 'BC', 'GL', 'BQ', 'AF', 'FD ', 'DI', 'AB', 'CC', 'EH', 'DA', 'FR', 'BN', 'AM', 'CR', 'GH', 'AY', 'EE', 'DF', 'FE', 'DE', 'CL', 'GF', 'FC', 'BP', 'CF', 'EB', 'EU', 'CD ', 'FI', 'GE', 'DH', 'AX', 'AH', 'BR', 'CW ', 'GI', 'AR']

Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.11494086086269004, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.1072447926134375, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.1755618442445653, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.1471342863573456, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 1ms/step
intermediate balanced log loss  tf.Tensor(0.19444498367297247, shape=(), dtype=float64)

At MIscore=0.001 Total features = 40
Feaures to use = ['DU', 'FL', 'BC', 'GL', 'BQ', 'AF', 'FD ', 'DI', 'AB', 'CC', 'EH', 'DA', 'FR', 'BN', 'AM', 'CR', 'GH', 'AY', 'EE', 'DF', 'FE', 'DE', 'CL', 'GF', 'FC', 'BP', 'CF', 'EB', 'EU', 'CD ', 'FI', 'GE', 'DH', 'AX', 'AH', 'BR', 'CW ', 'GI', 'AR', 'CS']

Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.11340922224124753, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 1ms/step
intermediate balanced log loss  tf.Tensor(0.11931587248815605, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.1869421876873575, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 2ms/step
intermediate balanced log loss  tf.Tensor(0.1360949628917216, shape=(), dtype=float64)
Assessing balanced log loss of  1
20/20 [==============================] - 0s 1ms/step
intermediate balanced log loss  tf.Tensor(0.19814547587576914, shape=(), dtype=float64)

Below this, the MIscores required are tiny and if any are included then all should probably be included.

## Added a voter 
Voter on 5 of 15 with the biggest value of the majority being taken on a unanimous vote, the second biggest on 4 or 3 of 5 as major or simple majority. 
An attempt was made to find a value to use when the vote is unanimous but in 
the end a binary search failed to produce better results than taking the highest
score and applying it for whichever way the vote ends up going unanimously.
It is not yet clear if the 4 or 3 of 5 case will occur, and if so what the best
weighted adjustment might be for a log loss function.

First try, it helped bring 0.22 to 0.18 on the public score. However, helping over an average may not be guaranteed and for now it appears that votes are always unanimous.

In [ ]:
# Generate the predictions for Greek's Gamma output only
# using ggmodel[10]

#modeltup = ggmodels[8]
#odel = modeltup[0]
#gpred_submission = model.predict(X_transformed) # for MLP and CNN
#gpred_submission = ggpred_submission.astype(np.float64)
#_gamma_np = np.array(y_gamma)
#intermediate_loss = balanced_log_loss(y_gamma_np, ggpred_submission)
#print ("Greek Gamma prediction balanced log loss", intermediate_loss)
#print(ggpred_submission)
#print(y_gamma_np)

ggpred_submission = np.zeros(len(X_transformed))

for bag in range(bag_num):
    for clf in lgbmodels[bag]:
        ggpred_submission += clf.predict_proba(X_transformed)[:,1] / n_fold / bag_num

print (ggpred_submission)


In [ ]:
#all_df = otrain.copy()

#y = pd.DataFrame(all_df['Class'],columns=['class_0'])
#y['class_1'] = all_df['Class'].fillna(0)
#y['class_0'] = y['class_1'] * -1 + 1

#print(y)

#X_transformed = all_df.copy()
#X_transformed.drop('Class', axis=1, inplace=True)
#X_transformed.drop('Id', axis=1, inplace=True)
#X_transformed = X_transformed[usefeats]
#X_transformed = X_transformed.astype(np.float64)

# run voter on models
modelno = 0
intermediate_loss = 100.0
y = y.astype(np.float64)
y_pred_ensemble = []
loss_sorted_models = []
for modeltup in models:
    model = modeltup[0]
    print ("Assessing balanced log loss of ", modelno+1)
    if modeltup[1] == 2:
        Xd = np.reshape(X_transformed, (X_transformed.shape[0], 1, X_transformed.shape[1])) # for LTSM
        pred_submission = model.predict(Xd)
        pred_submission = pred_submission.astype(np.float64)
        pred_submission = np.reshape(pred_submission, (pred_submission.shape[0], pred_submission.shape[2])) # only for LSTM
        intermediate_loss = balanced_log_loss_lstm(y, pred_submission)
    else:
        pred_submission = model.predict(X_transformed) # for MLP and CNN
        pred_submission = pred_submission.astype(np.float64)
        intermediate_loss = balanced_log_loss(y, pred_submission)
    y_pred_ensemble.append(pred_submission)
    print ("intermediate balanced log loss ", intermediate_loss)
    loss_sorted_models.append([intermediate_loss, model])
    
loss_sorted_models = sorted(loss_sorted_models, key=lambda x: x[0])
remodeled = []
for remodel in loss_sorted_models:
    for domodel in models:
        if remodel[1] == domodel[0]:
            remodeled.append([remodel[1], domodel[1]])
            
print("loss_sorted_models=", loss_sorted_models)
print("remodeled=", remodeled)
models = remodeled

def voted_predictions(X_data):
    # we ignore the first 5 and last 5 models since the likely contain outliers
    # initialize the possible predictions with the first model, and prep the rest
    pred_submission = []  # we will update this model according to votes & confidence
    pred_submission2 = []
    pred_submission3 = []
    pred_submission4 = []
    pred_submission5 = []
    basemodel = 5
    if models[basemodel][1] == 2:
        Xd = np.reshape(X_data, (X_data.shape[0], 1, X_data.shape[1])) # for LTSM
        pred_submission = models[basemodel][0].predict(Xd)
        pred_submission = pred_submission.astype(np.float64)
        pred_submission = np.reshape(pred_submission, (pred_submission.shape[0], pred_submission.shape[2])) # only for LSTM
    else:
        pred_submission = models[basemodel][0].predict(X_data) # for MLP and CNN
        pred_submission = pred_submission.astype(np.float64)
    # next model
    if models[basemodel+1][1] == 2:
        Xd = np.reshape(X_data, (X_data.shape[0], 1, X_data.shape[1])) # for LTSM
        pred_submission2 = models[basemodel+1][0].predict(Xd)
        pred_submission2 = pred_submission.astype(np.float64)
        pred_submission2 = np.reshape(pred_submission, (pred_submission.shape[0], pred_submission.shape[2])) # only for LSTM
    else:
        pred_submission2 = models[basemodel+1][0].predict(X_data) # for MLP and CNN
        pred_submission2 = pred_submission.astype(np.float64)
    # next model
    if models[basemodel+2][1] == 2:
        Xd = np.reshape(X_data, (X_data.shape[0], 1, X_data.shape[1])) # for LTSM
        pred_submission3 = models[basemodel+2][0].predict(Xd)
        pred_submission3 = pred_submission.astype(np.float64)
        pred_submission3 = np.reshape(pred_submission, (pred_submission.shape[0], pred_submission.shape[2])) # only for LSTM
    else:
        pred_submission3 = models[basemodel+2][0].predict(X_data) # for MLP and CNN
        pred_submission3 = pred_submission.astype(np.float64)
    if models[basemodel+3][1] == 2:
        Xd = np.reshape(X_data, (X_data.shape[0], 1, X_data.shape[1])) # for LTSM
        pred_submission4 = models[basemodel+3][0].predict(Xd)
        pred_submission4 = pred_submission.astype(np.float64)
        pred_submission4 = np.reshape(pred_submission, (pred_submission.shape[0], pred_submission.shape[2])) # only for LSTM
    else:
        pred_submission4 = models[basemodel+3][0].predict(X_data) # for MLP and CNN
        pred_submission4 = pred_submission.astype(np.float64)
    if models[basemodel+4][1] == 2:
        Xd = np.reshape(X_data, (X_data.shape[0], 1, X_data.shape[1])) # for LTSM
        pred_submission5 = models[basemodel+4][0].predict(Xd)
        pred_submission5 = pred_submission.astype(np.float64)
        pred_submission5 = np.reshape(pred_submission, (pred_submission.shape[0], pred_submission.shape[2])) # only for LSTM
    else:
        pred_submission5 = models[basemodel+4][0].predict(X_data) # for MLP and CNN
        pred_submission5 = pred_submission.astype(np.float64)
    # now that we have all 5 predictions for each test case find out the
    # class and if it is by 3 (simple), 4 (major), or 5 (unanimous) vote.
    class0votes5 = 0
    class0votes4 = 0
    class0votes3 = 0
    class1votes5 = 0
    class1votes4 = 0
    class1votes3 = 0        
    for row in range(len(pred_submission)):
        voteForClass0 = 0
        voteForClass1 = 0
        if pred_submission[row][0] >= 0.50:
            voteForClass0 += 1
        else:
            voteForClass1 += 1
        if pred_submission2[row][0] >= 0.50:
            voteForClass0 += 1
        else:
            voteForClass1 += 1
        if pred_submission3[row][0] >= 0.50:
            voteForClass0 += 1
        else:
            voteForClass1 += 1
        if pred_submission4[row][0] >= 0.50:
            voteForClass0 += 1
        else:
            voteForClass1 += 1
        if pred_submission5[row][0] >= 0.50:
            voteForClass0 += 1
        else:
            voteForClass1 += 1
        confidences = [pred_submission[row][0], pred_submission2[row][0], pred_submission3[row][0], pred_submission4[row][0], pred_submission5[row][0]]
        confidences.sort()
        if voteForClass0 == 5:
            class0votes5 += 1
            pred_submission[row][0] = confidences[4] #0.975 # confidences[4]
            pred_submission[row][1] = 1.0 - confidences[4] #0.025 # 1.0 - confidences[4]
        if voteForClass0 == 4:
            class0votes4 += 1
            pred_submission[row][0] = confidences[3]
            pred_submission[row][1] = 1.0 - confidences[3]
        if voteForClass0 == 3:
            class0votes3 += 1
            pred_submission[row][0] = confidences[3]
            pred_submission[row][1] = 1.0 - confidences[3]
        if voteForClass1 == 5:
            class1votes5 += 1
            pred_submission[row][0] = confidences[0] #0.025 #confidences[0]
            pred_submission[row][1] = 1.0 - confidences[0] #0.975 #1.0 - confidences[0]
        if voteForClass1 == 4:
            class1votes4 += 1
            pred_submission[row][0] = confidences[1]
            pred_submission[row][1] = 1.0 - confidences[1]
        if voteForClass1 == 3:
            class1votes3 += 1
            pred_submission[row][0] = confidences[1]
            pred_submission[row][1] = 1.0 - confidences[1]
    print(class0votes5,"unanimous votes cast for class 0")
    print(class0votes4,"majority votes cast for class 0")
    print(class0votes3,"simple votes cast for class 0")
    print(class1votes5,"unanimous votes cast for class 1")
    print(class1votes4,"majority votes cast for class 1")
    print(class1votes3,"simple votes cast for class 1")

    return pred_submission

def accuracy_metric(pred_submission, y):
    correctCount = 0
    for row in range(len(pred_submission)):
        if pred_submission[row][0] >= 0.5 and y[row][0] == 1:
            correctCount += 1
        if pred_submission[row][0] < 0.5 and y[row][0] == 0:
            correctCount += 1
    return correctCount/len(pred_submission)

# print out the voted on predictions
voted_pred = voted_predictions(X_transformed)
voted_balancedlogloss = balanced_log_loss(y, voted_pred)
print("+++++++++++++++++++++++ Voted average loss =", voted_balancedlogloss)
print("+++++++++++++++++++++++ Voted accuracy =", accuracy_metric(pred_submission, y))

# for now, will only work on MLP and CNN
if models[0][1] != 2:
    pred_submission = np.mean(y_pred_ensemble, axis=0)
    internal_avg_loss = balanced_log_loss(y, pred_submission)
    print("******************** Internal average loss =", internal_avg_loss)
    
total0 = 0
total1 = 0
correct0 = 0
correct1 = 0
pred0 = 0
pred1 = 0

for row in range(len(voted_pred)):
    if y[row][0] == 1:
        total0 += 1
    else:
        total1 += 1
    if y[row][0] >= 0.50:
        if pred_submission[row][0] >= 0.50:
            correct0 += 1
    if y[row][0] < 0.50:
        if pred_submission[row][0] < 0.50:
            correct1 += 1
    if pred_submission[row][0] >= 0.50:
        pred0 += 1
    if pred_submission[row][0] < 0.50:
        pred1 += 1
        
print ("Total true class0 =", total0)
print ("Total true class1 =", total1)
print ("Total correct class0 =", correct0)
print ("Total correct class1 =", correct1)
print ("Total predicted class0 =", pred0)
print ("Total predicted class1 =", pred1)


In [ ]:

total0 = 0
total1 = 0
correct0 = 0
correct1 = 0
pred0 = 0
pred1 = 0

for row in range(len(voted_pred)):
    if y[row][0] == 1:
        total0 += 1
    else:
        total1 += 1
    if y[row][0] >= 0.50:
        if ggpred_submission[row] < 0.50:
            correct0 += 1
    if y[row][0] < 0.50:
        if ggpred_submission[row] >= 0.50:
            correct1 += 1
    if ggpred_submission[row] < 0.50:
        pred0 += 1
    if ggpred_submission[row] >= 0.50:
        pred1 += 1
        
print ("Total true class0 =", total0)
print ("Total true class1 =", total1)
print ("Total correct class0 =", correct0)
print ("Total correct class1 =", correct1)
print ("Total predicted class0 =", pred0)
print ("Total predicted class1 =", pred1)


# code that analyzes the training test set to see if 
# using Greeks Gamma results might correct for false positives and negatives

# we are definitely seeing cases where the percentage confidence of any one 
# category in a row does not reach 50%. Rethink this to pick the highest 
# of the categories and see if using this category's class improves 
# results or not. YOU ARE HERE

def gghighscorer(ggrow):
    highscore = -1
    highscorer = 0
    for elem in range(len(ggrow)):
        if ggrow[elem]>highscore:
            highscorer = elem
            highscore = ggrow[elem]
    return highscorer

gghelper = [0, 0, 0, 0, 0, 0, 0, 0]
total0 = 0
total1 = 0

for row in range(len(voted_pred)):
    highscorer = gghighscorer(ggpred_submission[row])
    if y[row][0] == 1:
        total0 += 1
    else:
        total1 += 1
    if y[row][0] >= 0.50:
        if highscorer < 6:
            gghelper[highscorer] -= 1
        else:
            gghelper[highscorer] += 1
    else:
        if highscorer < 6:
            gghelper[highscorer] += 1
        else:
            gghelper[highscorer] -= 1

print ("gghelp order G,H,E,F,A,B,M,N =", gghelper)
print ("Total class0 =", total0)
print ("Total class1 =", total1)
print (y)

gghelper = [0, 0, 0, 0, 0, 0, 0, 0]

for row in range(len(voted_pred)):
    highscorer = gghighscorer(ggpred_submission[row])
    if voted_pred[row][0] >= 0.50: # can greeks gamma do better?
        if y[row][0] < 0.50: 
            if highscorer < 6:
                gghelper[highscorer] += 1
        else:
            if highscorer < 6:
                gghelper[highscorer] -= 1
    else: #voted_pred[row][0] < 0.50: # can greeks gamma do better?
        if y[row][0] >= 0.50:
            if highscorer >= 6:
                gghelper[highscorer] += 1
        else:
            if highscorer >= 6:
                gghelper[highscorer] -= 1

print ("gghelp order G,H,E,F,A,B,M,N =", gghelper)

In [ ]:
preds = np.zeros(len(test))

for bag in range(bag_num):
    for clf in lgbmodels[bag]:
            preds += clf.predict_proba(test[usefeats])[:,1] / n_fold / bag_num
            
my_submission = test[['Id']].copy()
my_submission['Class_0'] = 1-preds
my_submission['Class_1'] = preds


Submission

import math

print (test.head())

test_data_normal = test.copy()
test_data_normal.drop('Id', axis=1, inplace=True)
test_data_normal = test_data_normal[usefeats]
test_data_normal = np.array(test_data_normal)

if models[0][1] == 2:
    Xd_test = np.reshape(test_data_normal, (test_data_normal.shape[0], 1, test_data_normal.shape[1])) # only for LSTM
    y_pred = voted_predictions(Xd_test)
    #y_pred = models[2][0].predict(Xd_test)
else:
    y_pred = voted_predictions(test_data_normal)
    #y_pred = models[2][0].predict(test_data_normal)

my_submission = test.copy()
my_submission = my_submission[['Id']]

my_submission[['class_0']] = 0.0
my_submission[['class_1']] = 0.0
for row in range(len(y_pred)):
    if models[0][1] == 2:
        my_submission.loc[row,'class_0'] = y_pred[row][0][0]
        my_submission.loc[row,'class_1'] = y_pred[row][0][1]

    else:
        my_submission.loc[row,'class_0'] = y_pred[row][0]
        my_submission.loc[row,'class_1'] = y_pred[row][1]

for row in range(len(my_submission)): # sanity check so none are out of range
    if math.isnan(my_submission.loc[row,'class_0']):
        my_submission.loc[row,'class_1'] = 1.0-(1e-15)
        my_submission.loc[row,'class_0'] = 1e-15
        print ("Row", row, "isnan in class_0")
    if math.isnan(my_submission.loc[row,'class_1']):
        my_submission.loc[row,'class_1'] = 1.0-(1e-15)
        my_submission.loc[row,'class_0'] = 1e-15
        print ("Row", row, "isnan in class_1")
    if my_submission.loc[row,'class_0'] > 1.0-(1e-15):
        my_submission.loc[row,'class_0'] = 1.0-(1e-15)
        my_submission.loc[row,'class_1'] = 1e-15
        print ("Row", row, "is over 1 in class_0")
    if my_submission.loc[row,'class_1'] > 1.0-(1e-15):
        my_submission.loc[row,'class_1'] = 1.0-(1e-15)
        my_submission.loc[row,'class_0'] = 1e-15
        print ("Row", row, "is over 1 in class_1")
    if my_submission.loc[row,'class_0'] < 1e-15:
        my_submission.loc[row,'class_0'] = 1e-15
        my_submission.loc[row,'class_1'] = 1.0-(1e-15)
        print ("Row", row, "is too close to 0 in class_0")
    if my_submission.loc[row,'class_1'] < 1e-15:
        my_submission.loc[row,'class_1'] = 1e-15
        my_submission.loc[row,'class_0'] = 1.0-(1e-15)
        print ("Row", row, "is too close to 0 in class_1")
    # The attempt below to correct Class_1 to be Class_0 for Gamma cases
    # where  Gamma is M or N (last two categories) failed - loss 
    # went from .2 to .99
    #if my_submission.loc[row,'class_1'] > 0.50:
    #    # See if Greek's Gamma suggests swappinng to class_0
    #    highscorer = gghighscorer(ggpred_submission[row])
    #    if highscorer >= 6: # only consider M and N which map to class 0
    #        if gghelper[highscorer] > 0: # training concluded it might help
    #            holdClass0 = my_submission.loc[row,'class_0']
    #            my_submission.loc[row,'class_0'] = my_submission.loc[row,'class_1']
    #           my_submission.loc[row,'class_1'] = holdClass0


In [ ]:
if devrun:
    print(my_submission.head())


In [ ]:
my_submission.to_csv('submission.csv', index=False)
print ("Submission saved.")